<a href="https://colab.research.google.com/github/Nibalab/MedVision-ML/blob/main/Medvision_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
#we use this notebook to train the model of lung cancer detection
#https://github.com/BarriBarri20/Lung-cancer-detection-model-training/blob/main/lung_cancer.ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install tensorflow==2.12
import numpy as np
import base64
from PIL import Image
from io import BytesIO
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Load the pre-trained VGG16 model
model = load_model('/content/drive/MyDrive/vgg16 (1).hdf5')

def preprocess_image_from_base64(base64_str):
    """
    Decode a base64 string and preprocess it to match the input size and format expected by VGG16.
    """
    # Decode the base64 string
    image_data = base64.b64decode(base64_str)

    # Open the image from the decoded data
    image = Image.open(BytesIO(image_data))

    # Resize the image to the input size of VGG16
    image = image.resize((460, 460))

    # Convert the image to RGB if it's not already
    if image.mode != 'RGB':
        image = image.convert('RGB')

    # Convert the image to a numpy array
    image = img_to_array(image)

    # Normalize the image
    image = image / 255.0

    # Expand dimensions to match the model input shape
    image = np.expand_dims(image, axis=0)

    return image

def predict_image_from_base64(base64_str, model):
    """
    Predict the class of the image from a base64 string using the pre-trained model.
    """
    # Preprocess the image
    image = preprocess_image_from_base64(base64_str)

    # Make a prediction
    predictions = model.predict(image)

    return predictions

def interpret_predictions(predictions):
    """
    Interpret the model predictions.
    """
    print(predictions)
    # Uncomment and replace with actual class names for interpretation
    # class_labels = ['Class1', 'Class2', 'Class3']
    # predicted_class = np.argmax(predictions, axis=1)
    # return class_labels[predicted_class[0]]


In [3]:
!pip install flask-localtunnel
!pip install flask

!pip install flask-cors
!pip install pyngrok


!curl ifconfig.me


34.87.14.56

In [ ]:
from flask import Flask, jsonify, request
from flask_cors import CORS  # Import CORS
from pyngrok import ngrok

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

# Start ngrok when the app starts
ngrok.set_auth_token("2Nf4uxx5a5zO1BeoESMydWLXoZE_3hyQRFkEvgAQn6y8qZUsR")
public_url = ngrok.connect(5000)

print(" * ngrok tunnel: ", public_url)

@app.route("/detect-ct-scan-image", methods=["POST"])
def detectCancer():
    data = request.json
    base64_image = data.get('image_base64')

    if not base64_image:
        return jsonify({'error': 'No image_base64 provided'}), 400

    predictions = predict_image_from_base64(base64_image, model)

    try:
        values = predictions[0].tolist()  # Convert to list for easier handling
    except ValueError:
        return jsonify({'error': 'Invalid data format'}), 400

    # Check if the third element is the maximum
    if len(values) >= 3 and values[2] == max(values):
        result = "No cancer"
    else:
        result = "Cancer"

    return jsonify({'result': result}), 200

@app.route("/")
def test():
    return "test this"

if __name__ == '__main__':
    app.run()


 * ngrok tunnel:  NgrokTunnel: "https://69f0-34-87-14-56.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [18/Sep/2024 23:44:21] "OPTIONS /detect-ct-scan-image HTTP/1.1" 200 -


1/1 [==============================] - 2s 2s/step


INFO:werkzeug:127.0.0.1 - - [18/Sep/2024 23:44:24] "POST /detect-ct-scan-image HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Sep/2024 23:45:12] "OPTIONS /detect-ct-scan-image HTTP/1.1" 200 -


1/1 [==============================] - 2s 2s/step


INFO:werkzeug:127.0.0.1 - - [18/Sep/2024 23:45:15] "POST /detect-ct-scan-image HTTP/1.1" 200 -


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 73.7 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.0
    Uninstalling tensorboard-2.17.0:
      Successfully uninstalled tensorboard-2.17.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.0
    Uninstalling tensorflow-2.17.0:
      Successfully uninstalled tensorflow-2.17.0
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-p

AttributeError: Exception encountered when calling Flatten.call().

[1m'list' object has no attribute 'shape'[0m

Arguments received by Flatten.call():
  • args=(['<KerasTensor shape=(None, 512), dtype=float32, sparse=False, name=keras_tensor_104>'],)
  • kwargs=<class 'inspect._empty'>